In [1]:
import importlib
import data.VCTK
import model.model
import model.trainer
import torch

importlib.reload(data.VCTK) # prevent cache when making changes
importlib.reload(model.trainer) # prevent cache when making changes
importlib.reload(model.model) # prevent cache when making changes

<module 'model.model' from '/home/azureuser/dom/Project/model/model.py'>

In [2]:
dataset = data.VCTK.VCTKDataset("VCTK-Corpus-smaller/")

device = torch.device("cuda")
myModel = model.model.SpectrogramModel().to(device)

In [33]:
x = dataset[0][2]
x = x[0].unsqueeze(0)
print(x.shape)
x = x.repeat(50, 1, 1)
print(x.shape)

torch.Size([1, 80, 129])
torch.Size([50, 80, 129])


In [3]:
sum([len(x) for x in list(myModel.parameters())])

129184

In [53]:
torch.cuda.empty_cache()

In [5]:
t = torch.cuda.get_device_properties(0).total_memory
r = torch.cuda.memory_reserved(0)
a = torch.cuda.memory_allocated(0)
f = r-a  # free inside reserved
def reasonable(n):
    return n/(1000**2)
print(reasonable(t),reasonable(r),reasonable(a),reasonable(f))

11996.954624 555.74528 295.996928 259.748352


In [22]:
del x

In [6]:
before = torch.cuda.memory_allocated(0)
x = dataset[0][2][0,:,:].to(device).repeat(100,1,1)
after = torch.cuda.memory_allocated(0)
print(f"Took {reasonable(after-before):1f}MB")
print(x.shape)

Took 4.128256MB
torch.Size([100, 80, 129])


/home/azureuser/dom/Project/data/VCTK.py:62: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1634272168786/work/torch/csrc/utils/tensor_new.cpp:201.)
  return text, torch.tensor(clips), torch.tensor(spectros)


In [8]:
importlib.reload(model.trainer) # prevent cache when making changes
myTrainer = model.trainer.Trainer(myModel, device)

In [9]:
import tqdm.notebook as tqdm
for i, _ in tqdm.tqdm(list(enumerate(range(0,100)))):
    myTrainer.train_step(x, x)

  0%|          | 0/100 [00:00<?, ?it/s]

tensor(1.5168, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5275, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5516, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5807, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5696, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5472, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5355, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5308, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5179, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5116, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.5048, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4945, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4866, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4849, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4686, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4642, device='cuda:0', grad_fn=<L1LossBackward0>)
tensor(1.4439, device='cuda:0', grad_fn=<L1LossBackward0

In [16]:
import pathlib
import os.path
checkpoint_dir = pathlib.Path("checkpoints")
checkpoint_dir.mkdir(exist_ok=True)
model_path = checkpoint_dir / "model1.pth"
if os.path.exists(model_path):
    raise Exception("Won't overwrite existing models")
else:
    torch.save(myModel.state_dict(), model_path)